In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import compress

from datetime import datetime
from dateutil.parser import parse

import math
import os
import copy

import pickle

In [2]:
# Reading the text file
indy_data = pd.read_csv('indy_opioid-geo.csv', engine = 'python')

In [3]:
indy_data.head()

,Case_Number,Year,Sex,Race,DOD,Age,Military,Employment,Education,Marital_Status,...,MPreDirectional,MName,MSuffix,MCity,MState,MZip,QualityInjury,FeatureMatchingHierarchyNotesInjury,LatitudeInjury,LongitudeInjury
0,100064,2010,1,1,1/14/10,20,0.0,2.0,2.0,1,...,NaN,HARLAN,ST,Indianapolis,IN,46203,ExactParcelCentroidPoint,NaN,39.746800,-86.125646
1,100076,2010,1,2,1/16/10,42,0.0,1.0,2.0,1,...,N,DENNY,ST,Indianapolis,IN,46218,ExactParcelCentroidPoint,NaN,39.816930,-86.101735
2,100079,2010,0,1,1/17/10,28,0.0,1.0,4.0,1,...,NaN,STONEROOT,PL,Indianapolis,IN,46203,ExactParcelCentroidPoint,NaN,39.726285,-86.063188
3,100092,2010,1,2,1/20/10,47,0.0,1.0,6.0,2,...,NaN,MURPHYS,LNDG,Indianapolis,IN,46220,USPSZipAreaCentroid,NaN,39.867992,-86.108018
4,100095,2010,1,1,1/21/10,47,0.0,1.0,4.0,2,...,NaN,SHILO,RD,Unionville,IN,47468,AddressRangeInterpolation,NaN,39.651268,-86.167904


In [4]:
drug_test = indy_data.iloc[:, 26:206]

In [5]:
drug_test = drug_test.drop(['Illicits','Drugs_of_Interest', 'Any_Heroin', 'Any_Fentanyl',
                            'Any_Illicit_Opioid', 'Any_Prescription_Opioid', 'Any_Opioid',
                            'Any_Benzodiazepine', 'Any_Cocaine', 'Any_Methamphetamine',
                            'Drug_Concentrations'], axis=1)

In [6]:
drug_test[(drug_test.sum(axis=1)) > 1].head()

,Alcohols,Ethanol,Acetone,Isopropanol,Amphetamines,Amphetamine,Methamphetamine,Ephedrine,MDMA,MDA,...,Fentanyl_Result,Norfentanyl_Result,Oxycodone_Result,Hydrocodone_Result,Oxymorphone_Result,Hydromorphone_Result,Dihydrocodeine_Result,Cocaine_Result,Benzoylecgonine_Result,Methadone_Result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# gets drug columns equal to one for each row
def getDrugs(row):
    row_preds = []
    for col_name, value in row.items():
        if value == 1:
            row_preds.append(col_name)
    return(row_preds)

In [8]:
indy_data['drug_preds'] = drug_test.apply(getDrugs, axis = 1)

In [9]:
indy_data['drug_preds']

0                  [Benzodiazepine, Opioids, Oxymorphone]
1                            [Alcohols, Ethanol, Cocaine]
2                            [Alcohols, Ethanol, Cocaine]
3       [Benzodiazepine, Opioids, Hydrocodone, Hydromo...
4                                               [Cocaine]
                              ...                        
1564                                  [Alcohols, Ethanol]
1565    [Alcohols, Ethanol, Stimulants, Caffeine, Nico...
1566    [Alcohols, Ethanol, Benzodiazepine, Alprazolam...
1567    [Benzoylecgonine, Opioids, Fentanyl, Norfentanyl]
1568    [Benzodiazepine, Alprazolam, 7AminoClonazepam,...
Name: drug_preds, Length: 1569, dtype: object

## Co-Occurance

In [10]:
#Get the list unique drugs for each buyer
drug_series = indy_data['drug_preds'].tolist()

In [11]:
from collections import Counter
from itertools import combinations

counter  = Counter()

In [12]:
#find the most common pairings of each list
for sub in drug_series:
    if len(drug_series) < 2:
        continue
    sub.sort()
    for comb in combinations(sub,2):
        counter[comb] += 1

In [13]:
#Print the most common pairings
most_common = counter.most_common()

most_common

[(('Morphine', 'Opioids'), 723),
 (('Benzodiazepine', 'Opioids'), 606),
 (('6_MAM', 'Opioids'), 565),
 (('6_MAM', 'Morphine'), 562),
 (('Codeine', 'Opioids'), 533),
 (('6_MAM', 'Codeine'), 531),
 (('6_MAM', 'Heroin_from_combo'), 531),
 (('Codeine', 'Heroin_from_combo'), 531),
 (('Codeine', 'Morphine'), 531),
 (('Heroin_from_combo', 'Morphine'), 531),
 (('Heroin_from_combo', 'Opioids'), 531),
 (('Alcohols', 'Ethanol'), 365),
 (('Benzodiazepine', 'Morphine'), 341),
 (('Fentanyl', 'Opioids'), 337),
 (('Hydrocodone', 'Opioids'), 264),
 (('6_MAM', 'Benzodiazepine'), 264),
 (('Alcohols', 'Opioids'), 261),
 (('Ethanol', 'Opioids'), 261),
 (('Benzodiazepine', 'Codeine'), 251),
 (('Benzodiazepine', 'Heroin_from_combo'), 251),
 (('Hydromorphone', 'Opioids'), 239),
 (('Cocaine', 'Opioids'), 239),
 (('Opioids', 'Oxycodone'), 234),
 (('Amphetamines', 'Opioids'), 204),
 (('Opioids', 'Oxymorphone'), 200),
 (('Benzodiazepine', 'Hydrocodone'), 189),
 (('Amphetamines', 'Methamphetamine'), 173),
 (('Fent

In [14]:
# Read in data (from pickle file)
file = open('market_co','rb')
market_co = pickle.load(file)

In [15]:
# naming columns of the Dream market co-occurences dataframe
market_co.columns = ['drug_co','occurs']

In [16]:
#create Indy co-occurence dataframe
indy_co = pd.DataFrame(most_common)
indy_co.columns = ['drug_co', 'occurs']

In [17]:
#lowering a tuple list
def tokenize(tup):
    lowered = tuple(item.lower() for item in tup)
    return(lowered)

In [18]:
#lowering the drug co-occurences in Indy
indy_co['drug_co'] = indy_co['drug_co'].apply(tokenize)

In [19]:
#save as pickle
import pickle
file=open('indy_co' , 'wb')
pickle.dump(indy_co, file)
file.close()

In [20]:
#Export as csv
indy_data.to_csv('indy_drugs.csv')